In [3]:
import sys, os

src = r"C:\Users\user\Desktop\Coding mo\AutoML\src" # copy the location of the folder that has paths.py file using (ctrl + shift + C ) and paste it here    
sys.path.append(src)

from paths import *

get_paths()

sys.path.append(SRC_PATH)

from pipeline import *


In [2]:
# ---------------- Task 1+9 ----------------

from autogluon.tabular import TabularPredictor
import torch


def loop_dfs_and_evaluate_autogluon():
    for dataset_name in data_dict_classification_only.keys():
        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = dataset_name.replace(".csv", "")

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)
            
            try:
                torch.manual_seed(seed)
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            except:
                pass

            df_train = splits_dict[dataset_name]['train']
            df_test = splits_dict[dataset_name]['test']
            task_type = tasks_dict_classification_only[dataset_name]

            start_time = time.time()
            predictor = TabularPredictor(
                label=TARGET_COLS[dataset_name],
                problem_type=task_type,
                path=os.path.join(AUTOML_METRICS_PATH, f"autogluon_{dataset_base}_{seed}")
            ).fit(
                df_train,
                time_limit=TIME_BUDGET,
                memory_limit=MEMORY_LIMIT
            )
            training_runtime_sec = round(time.time() - start_time, 3)
            
            infer_start = time.time()
            results = evaluate_and_save_results_general(df_train, df_test, predictor, dataset_name, framework='autogluon')
            inference_time_per_sample = round((time.time() - infer_start) / len(df_test), 6)
            cpu_usage_percent = psutil.cpu_percent(interval=None)

            model_dir = predictor.path
            total_bytes = 0
            for root, dirs, files in os.walk(model_dir):
                for f in files:
                    fp = os.path.join(root, f)
                    total_bytes += os.path.getsize(fp)
            model_size_mb = round(total_bytes / (1024 * 1024), 3)

            memory_usage_mb = get_memory_usage_mb()

            runtime_memory_rows.append({
                "dataset": dataset_name,
                "seed": seed,
                "training_runtime_sec": training_runtime_sec,
                "model_size_MB": model_size_mb,
                "memory_usage_MB": memory_usage_mb,
                "inference_time_per_sample_sec": inference_time_per_sample,
                "cpu_usage_percent": cpu_usage_percent
            })

            dataset_summary.append({
                'dataset': dataset_name, 
                'seed': seed, 
                **results, 
                'runtime': training_runtime_sec
            })

            del predictor
            gc.collect()

        output_path = os.path.join(AUTOML_METRICS_PATH, f"autogluon_{dataset_base}_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(output_path, index=False)

        task1_output_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"autogluon_{dataset_base}_task1_runtime_memory_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task1_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"autogluon_{dataset_base}_runtime_memory_metrics_in_json.json")

        
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

loop_dfs_and_evaluate_autogluon()











Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       6.24 GB / 31.92 GB (19.5%)
Disk Space Avail:   106.22 GB / 476.13 GB (22.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='

In [3]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace(".csv", "")
    csv_file1  = os.path.join(AUTOML_METRICS_PATH, f"autogluon_{dataset_base}_summary.csv")
    csv_file2  = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"autogluon_{dataset_base}_task1_runtime_memory_metrics.csv")
    json_file1 = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"autogluon_{dataset_base}_runtime_memory_metrics_in_json.json")

    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_task1_runtime_memory_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)

    # print(f"JSON for {dataset_base}: autogluon_{dataset_base}_runtime_memory_metrics_in_json.json")
    # df3 = pd.read_json(json_file1)
    # display(df3)



CSV for modeldata: autogluon_modeldata_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,0.927417,0.927465,0.927521,0.927417,37.111
1,modeldata.csv,123,0.927417,0.927465,0.927521,0.927417,39.331


CSV for modeldata: autogluon_modeldata_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,37.111,1089.093,4772.992,0.000034,49.1
1,modeldata.csv,123,39.331,1775.176,4741.988,0.000071,80.5


CSV for titanic: autogluon_titanic_summary.csv


...

,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,8.201,4.409,4934.707,0.000734,97.7
1,train.csv,123,7.873,4.409,4935.156,0.000667,97.3


CSV for wine: autogluon_wine_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1.0,1.0,1.0,1.0,7.007
1,wine.csv,123,1.0,1.0,1.0,1.0,6.441


CSV for wine: autogluon_wine_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,7.007,5.513,4935.832,0.001544,67.6
1,wine.csv,123,6.441,5.513,4935.992,0.001835,72.8


In [4]:
# ---------------- Task 3+9 ----------------

from autogluon.tabular import TabularPredictor
import torch

def evaluate_cv_folds_autogluon(folds_dict):
    for dataset_name, folds_data in folds_dict.items():
        task_type = tasks_dict_classification_only[dataset_name]
        target_col = TARGET_COLS[dataset_name]

        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = dataset_name.replace(".csv", "")

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)
            try:
                torch.manual_seed(seed)
                torch.cuda.empty_cache()
            except:
                pass

            fold_num = 1
            for fold_name, fold_splits in folds_data.items():
                df_train = fold_splits['train']
                df_val   = fold_splits['val']

                X_val = df_val.drop(columns=[target_col])
                y_val = df_val[target_col]

                temp_dir = tempfile.mkdtemp()

                start_time = time.time()
                predictor = TabularPredictor(
                    label=target_col,
                    problem_type=task_type,
                    path=temp_dir
                ).fit(
                    df_train,
                    time_limit=CV_TIME_BUDGET,
                    memory_limit=CV_MEMORY_LIMIT
                )
                training_runtime_sec = round(time.time() - start_time, 3)

                infer_start = time.time()
                preds = predictor.predict(X_val)
                results = evaluate_and_save_results_general(df_train, df_val, predictor, dataset_name, framework='autogluon')
                inference_time_per_sample = round((time.time() - infer_start) / len(X_val), 6)
                cpu_usage_percent = psutil.cpu_percent(interval=None)

                total_bytes = 0
                for root, dirs, files in os.walk(temp_dir):
                    for f in files:
                        total_bytes += os.path.getsize(os.path.join(root, f))
                model_size_mb = round(total_bytes / (1024 * 1024), 3)

                memory_usage_mb = get_memory_usage_mb()

                dataset_summary.append({
                    'dataset': dataset_name,
                    'seed': seed,
                    'fold': fold_num,
                    **results,
                    'runtime': training_runtime_sec
                })

                runtime_memory_rows.append({
                    "dataset": dataset_name,
                    "seed": seed,
                    "fold": fold_num,
                    "training_runtime_sec": training_runtime_sec,
                    "model_size_MB": model_size_mb,
                    "memory_usage_MB": memory_usage_mb,
                    "inference_time_per_sample_sec": inference_time_per_sample,
                    "cpu_usage_percent": cpu_usage_percent
                })

                fold_num += 1
                shutil.rmtree(temp_dir, ignore_errors=True)
                del predictor
                gc.collect()

        output_path = os.path.join(CV_FOLDS_METRICS_PATH, f"autogluon_{dataset_base}_cv_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(output_path, index=False)

        task3_output_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"autogluon_{dataset_base}_task3_runtime_memory_cv_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task3_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"autogluon_{dataset_base}_runtime_memory_metrics_cv_in_json.json")
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

evaluate_cv_folds_autogluon(folds_dict)





Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       4091.14 GB / 4096.00 GB (99.9%)
Disk Space Avail:   106.22 GB / 476.13 GB (22.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	pres

In [5]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    csv_file1 = os.path.join(CV_FOLDS_METRICS_PATH, f"autogluon_{dataset_base}_cv_summary.csv")
    csv_file2 = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"autogluon_{dataset_base}_task3_runtime_memory_cv_metrics.csv")
    json_file1 = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"autogluon_{dataset_base}_runtime_memory_metrics_cv_in_json.json")

    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_cv_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_task3_runtime_memory_cv_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)
    
    # print(f"JSON for {dataset_base}: autogluon_{dataset_base}_runtime_memory_cv_metrics_in_json.json")
    # df3 = pd.read_json(json_file1)
    # display(df3)
    

CSV for modeldata: autogluon_modeldata_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,0.925412,0.925368,0.925331,0.925412,21.989
1,modeldata.csv,42,2,0.924055,0.923964,0.923901,0.924055,22.841
2,modeldata.csv,42,3,0.927263,0.927413,0.927654,0.927263,21.609
3,modeldata.csv,42,4,0.923900,0.924081,0.924380,0.923900,24.231
4,modeldata.csv,42,5,0.929885,0.929859,0.929836,0.929885,21.245
5,modeldata.csv,123,1,0.925412,0.925368,0.925331,0.925412,23.212
6,modeldata.csv,123,2,0.924055,0.923964,0.923901,0.924055,22.261
7,modeldata.csv,123,3,0.927263,0.927413,0.927654,0.927263,24.715
8,modeldata.csv,123,4,0.923900,0.924081,0.924380,0.923900,22.113
9,modeldata.csv,123,5,0.929885,0.929859,0.929836,0.929885,22.822


CSV for modeldata: autogluon_modeldata_task3_runtime_memory_cv_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,1,21.989,559.316,5150.520,0.000012,90.5
1,modeldata.csv,42,2,22.841,558.059,5151.535,0.000014,91.4
2,modeldata.csv,42,3,21.609,563.095,5153.730,0.000011,96.5
3,modeldata.csv,42,4,24.231,557.381,5162.766,0.000014,95.0
4,modeldata.csv,42,5,21.245,563.110,5162.625,0.000011,98.5
5,modeldata.csv,123,1,23.212,559.316,5165.277,0.000012,92.2
6,modeldata.csv,123,2,22.261,558.059,5164.562,0.000012,88.5
7,modeldata.csv,123,3,24.715,563.095,5164.957,0.000011,95.3
8,modeldata.csv,123,4,22.113,557.381,5161.734,0.000014,87.6
9,modeldata.csv,123,5,22.822,563.110,5165.398,0.000010,88.4


CSV for titanic: autogluon_titanic_cv_summary.csv


...

,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,1,6.456,5.841,4720.391,0.001400,72.0
1,train.csv,42,2,6.057,3.679,4720.320,0.004243,69.7
2,train.csv,42,3,6.189,3.001,4720.617,0.001207,66.1
3,train.csv,42,4,6.211,4.144,4721.281,0.004459,72.6
4,train.csv,42,5,6.108,3.292,4721.480,0.001145,66.4
5,train.csv,123,1,6.510,5.841,4721.254,0.001367,70.8
6,train.csv,123,2,6.313,3.679,4720.988,0.004611,76.6
7,train.csv,123,3,6.525,3.001,4721.289,0.001100,69.0
8,train.csv,123,4,6.174,4.144,4721.195,0.004401,66.9
9,train.csv,123,5,6.051,3.292,4721.414,0.001169,69.5


CSV for wine: autogluon_wine_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,0.972222,0.972369,0.974747,0.972222,6.174
1,wine.csv,42,2,1.000000,1.000000,1.000000,1.000000,6.246
2,wine.csv,42,3,1.000000,1.000000,1.000000,1.000000,6.450
3,wine.csv,42,4,0.971429,0.971471,0.973626,0.971429,6.786
4,wine.csv,42,5,1.000000,1.000000,1.000000,1.000000,6.777
5,wine.csv,123,1,0.972222,0.972369,0.974747,0.972222,6.681
6,wine.csv,123,2,1.000000,1.000000,1.000000,1.000000,6.444
7,wine.csv,123,3,1.000000,1.000000,1.000000,1.000000,6.647
8,wine.csv,123,4,0.971429,0.971471,0.973626,0.971429,6.981
9,wine.csv,123,5,1.000000,1.000000,1.000000,1.000000,7.059


CSV for wine: autogluon_wine_task3_runtime_memory_cv_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,1,6.174,4.408,4721.789,0.003797,67.6
1,wine.csv,42,2,6.246,5.012,4721.887,0.000695,70.0
2,wine.csv,42,3,6.450,4.468,4721.273,0.001111,68.9
3,wine.csv,42,4,6.786,6.647,4722.598,0.003387,76.2
4,wine.csv,42,5,6.777,4.741,4722.082,0.001115,92.6
5,wine.csv,123,1,6.681,4.408,4722.672,0.003723,94.1
6,wine.csv,123,2,6.444,5.012,4722.648,0.000639,94.6
7,wine.csv,123,3,6.647,4.468,4722.785,0.001056,92.6
8,wine.csv,123,4,6.981,6.647,4722.625,0.004516,94.6
9,wine.csv,123,5,7.059,4.741,4722.852,0.001086,94.2


In [2]:
# ---------------- Task 6 ----------------

from autogluon.tabular import TabularPredictor

def evaluate_pipeline_folds_times_autogluon(leakage_free_final_output):
    for dataset_name, folds in leakage_free_final_output.items():
        dataset_base = dataset_name.replace('.csv', '')

        dataset_summary = []
        time_log_rows = []

        target_col = TARGET_COLS[dataset_name]

        for seed in RANDOM_SEEDS:

            np.random.seed(seed)
            random.seed(seed)

            for fold_name, split in folds.items():

                df_train = split['train'].copy()
                df_val = split['val'].copy()
                pipe_times = split['pipeline_time']
                pipe_times_copy = pipe_times.copy()
                pipe_times_copy["total_time"] = sum(pipe_times_copy.values())

                X_train = df_train.drop(columns=[target_col])
                y_train = df_train[target_col]

                X_val = df_val.drop(columns=[target_col])
                y_val = df_val[target_col]


   

                temp_dir = tempfile.mkdtemp()
                start = time.time()

                df_train_proc = X_train.copy()
                df_train_proc[target_col] = y_train.values

                predictor = TabularPredictor(
                    label=target_col,
                    problem_type=tasks_dict_classification_only[dataset_name],
                    path=temp_dir
                ).fit(df_train_proc, time_limit=CV_TIME_BUDGET)

                preds = predictor.predict(X_val)
                runtime = round(time.time() - start, 3)
                fold_idx = int(fold_name.split("_")[1])

                best_model_raw = predictor.leaderboard(silent=True).iloc[0]["model"]
                best_model = f"autogluon - {best_model_raw}"

                try:
                    model_obj = predictor._trainer.load_model(best_model_raw)
                    generate_shap_explanations(
                        model=model_obj,
                        X=X_val,
                        model_name=best_model_raw.replace("/", "_"),
                        save_base_path=AUTOGUON_SHAP_PATH
                    )
                except:
                    pass
                    
                dataset_summary.append({
                    "dataset": dataset_name,
                    "seed": seed,
                    "fold": fold_idx,
                    "model_name": best_model,
                    "accuracy": (accuracy_score(y_val, preds)),
                    "f1": (f1_score(y_val, preds, average='weighted')),
                    "precision": (precision_score(y_val, preds, average='weighted')),
                    "recall": (recall_score(y_val, preds, average='weighted')),
                    "runtime": runtime
                })

                row = {"dataset": dataset_name, "fold": fold_idx, "model_name": best_model}
                row.update(pipe_times_copy)
                time_log_rows.append(row)

                shutil.rmtree(temp_dir, ignore_errors=True)

        df_summary = pd.DataFrame(dataset_summary).drop_duplicates()
        df_times = pd.DataFrame(time_log_rows).drop_duplicates()
        
        print(f"\nSummary for dataset: {dataset_name}")
        display(df_summary)

        df_summary = df_summary.sort_values(["dataset", "seed", "fold"])
        df_times = df_times.sort_values(["dataset", "fold"])

        summary_path = os.path.join(
            PIPELINE_DATAFRAMES_METRICS_PATH,
            f"autogluon_{dataset_base}_pipeline_summary.csv")
        df_summary.to_csv(summary_path, index=False)

        time_log_path = os.path.join(
            PIPELINE_LOG_TIMES_PATH,
            f"autogluon_{dataset_base}_pipeline_time_log.csv")
        df_times.to_csv(time_log_path, index=False)


evaluate_pipeline_folds_times_autogluon(run_pipeline_on_folds_with_control(pipeline_data_dict))


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       9.13 GB / 31.92 GB (28.6%)
Disk Space Avail:   92.34 GB / 476.13 GB (19.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='h


Summary for dataset: modeldata.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,autogluon - LightGBM,0.919218,0.918952,0.918885,0.919218,11.265
1,modeldata.csv,42,2,autogluon - WeightedEnsemble_L2,0.922759,0.922470,0.922437,0.922759,10.848
2,modeldata.csv,123,1,autogluon - WeightedEnsemble_L2,0.918551,0.918336,0.918251,0.918551,12.173
3,modeldata.csv,123,2,autogluon - LightGBMXT,0.922759,0.922470,0.922437,0.922759,11.565


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       4.48 GB / 31.92 GB (14.0%)
Disk Space Avail:   92.34 GB / 476.13 GB (19.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='h


Summary for dataset: titanic.csv


...


Summary for dataset: train.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,train.csv,42,1,autogluon - CatBoost,0.946667,0.946581,0.948470,0.946667,4.816
1,train.csv,42,2,autogluon - CatBoost,0.920000,0.918846,0.919656,0.920000,4.962
2,train.csv,123,1,autogluon - CatBoost,0.946667,0.946581,0.948470,0.946667,4.805
3,train.csv,123,2,autogluon - CatBoost,0.920000,0.918846,0.919656,0.920000,4.742


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       4.83 GB / 31.92 GB (15.1%)
Disk Space Avail:   92.34 GB / 476.13 GB (19.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='h


Summary for dataset: wine.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,autogluon - CatBoost,0.955056,0.954885,0.955118,0.955056,5.387
1,wine.csv,42,2,autogluon - LightGBMLarge,0.898876,0.898967,0.899390,0.898876,5.186
2,wine.csv,123,1,autogluon - CatBoost,0.955056,0.954885,0.955118,0.955056,5.177
3,wine.csv,123,2,autogluon - LightGBMLarge,0.898876,0.898967,0.899390,0.898876,5.186


In [3]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    
    csv_file_1 = os.path.join(PIPELINE_DATAFRAMES_METRICS_PATH, f"autogluon_{dataset_base}_pipeline_summary.csv")
    csv_file_2 = os.path.join(PIPELINE_LOG_TIMES_PATH, f"autogluon_{dataset_base}_pipeline_time_log.csv")
    
    df1 = pd.read_csv(csv_file_1)
    df2 = pd.read_csv(csv_file_2)
    
    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_pipeline_summary.csv")
    display(df1) 
    print(f"CSV for {dataset_base}: autogluon_{dataset_base}_pipeline_time_log.csv")
    display(df2) 



CSV for modeldata: autogluon_modeldata_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,autogluon - LightGBM,0.919218,0.918952,0.918885,0.919218,11.265
1,modeldata.csv,42,2,autogluon - WeightedEnsemble_L2,0.922759,0.922470,0.922437,0.922759,10.848
2,modeldata.csv,123,1,autogluon - WeightedEnsemble_L2,0.918551,0.918336,0.918251,0.918551,12.173
3,modeldata.csv,123,2,autogluon - LightGBMXT,0.922759,0.922470,0.922437,0.922759,11.565


CSV for modeldata: autogluon_modeldata_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,selector,total_time
0,modeldata.csv,1,autogluon - LightGBM,0.525178,0.182331,0.013367,0.084668,0.059857,0.024257,0.889658
1,modeldata.csv,1,autogluon - WeightedEnsemble_L2,0.525178,0.182331,0.013367,0.084668,0.059857,0.024257,0.889658
2,modeldata.csv,2,autogluon - WeightedEnsemble_L2,0.518440,0.202923,0.020153,0.097889,0.080931,0.046473,0.966809
3,modeldata.csv,2,autogluon - LightGBMXT,0.518440,0.202923,0.020153,0.097889,0.080931,0.046473,0.966809


CSV for titanic: autogluon_titanic_pipeline_summary.csv


...

,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,train.csv,1,autogluon - CatBoost,0.002004,0.0,0.001002,0.000000,0.0,0.0,0.0,0.003006
1,train.csv,2,autogluon - CatBoost,0.002004,0.0,0.001002,0.001003,0.0,0.0,0.0,0.004009


CSV for wine: autogluon_wine_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,autogluon - CatBoost,0.955056,0.954885,0.955118,0.955056,5.387
1,wine.csv,42,2,autogluon - LightGBMLarge,0.898876,0.898967,0.899390,0.898876,5.186
2,wine.csv,123,1,autogluon - CatBoost,0.955056,0.954885,0.955118,0.955056,5.177
3,wine.csv,123,2,autogluon - LightGBMLarge,0.898876,0.898967,0.899390,0.898876,5.186


CSV for wine: autogluon_wine_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,wine.csv,1,autogluon - CatBoost,0.003201,0.0,0.000000,0.001031,0.0,0.001009,0.0,0.005240
1,wine.csv,2,autogluon - LightGBMLarge,0.003007,0.0,0.001002,0.001002,0.0,0.001002,0.0,0.006013
